In [1]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
import os

azure_search_key = os.getenv('AZURE_SEARCH_KEY')
azure_search_endpoint = os.getenv('AZURE_SEARCH_ENDPOINT')

credential = AzureKeyCredential(azure_search_key)
index_client = SearchIndexClient(endpoint=azure_search_endpoint, credential=credential)

In [2]:

from langchain_openai import AzureOpenAIEmbeddings

azure_embeddings_deployment = os.getenv('AZURE_EMBEDDINGS_DEPLOYMENT')
azure_oai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_oai_key = os.getenv('AZURE_OPENAI_API_KEY')

embeddings = AzureOpenAIEmbeddings(azure_deployment=azure_embeddings_deployment,
                                       azure_endpoint=azure_oai_endpoint,
                                       api_key=azure_oai_key,
                                       api_version="2023-05-15")

In [3]:
ss = embeddings.embed_query("hey there")

len(ss)

1536

In [10]:
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SearchField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
     VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,

)

embedding_function = embeddings.embed_query

fields = [
    SimpleField(name="docId", type=SearchFieldDataType.String, key=True),
    SearchableField(name="exam", type=SearchFieldDataType.String),
    SearchableField(name="module", type=SearchFieldDataType.String),
    SearchableField(name="unit", type=SearchFieldDataType.String),
    SearchableField(name="topic", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="links", type=SearchFieldDataType.Collection(SearchFieldDataType.String), ),
    SearchField(name="images", type=SearchFieldDataType.Collection(SearchFieldDataType.ComplexType),
                    fields=[
                        SearchField(name="image_name", type=SearchFieldDataType.String, searchable=True),
                        SearchField(name="image_src", type=SearchFieldDataType.String, searchable=True),
                        SearchField(name="image_alt", type=SearchFieldDataType.String, searchable=True)
                    ]),
    SearchField(name="videos", type=SearchFieldDataType.Collection(SearchFieldDataType.ComplexType),
                    fields=[
                        SearchField(name="name", type=SearchFieldDataType.String, searchable=True),
                        SearchField(name="video_src", type=SearchFieldDataType.String, searchable=True)
                    ]),
    SearchableField(name="exercise_url", type=SearchFieldDataType.String),
   SearchField(name="content_vector",
                    type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True,
                    vector_search_dimensions=1536,
                    vector_search_profile_name = "myHnswProfile",  
                    vector_search_configuration="default"),
]

vector_search = VectorSearch(algorithms=[HnswAlgorithmConfiguration(name="myHnsw")],
                                 profiles=[VectorSearchProfile(name="myHnswProfile", algorithm_configuration_name="myHnsw")])


index = SearchIndex(name="exam-simulator-idx", fields=fields, vector_search=vector_search)
result = index_client.create_or_update_index(index)
print(result.name)

vector_search_configuration is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored


exam-simulator-idx


In [6]:
documents = [
    {
        "DocId":"1",
        "Exam":"AI-102 AI Engineer",
        "Module": "Develop generative AI apps in Azure",
        "Topic": "Plan and prepare to develop AI solutions on Azure",
        "content": """Introduction
Completed
100 XP
1 minute
The growth in the use of artificial intelligence (AI) in general, and generative AI in particular means that developers are increasingly required to create comprehensive AI solutions. These solutions need to combine machine learning models, AI services, prompt engineering solutions, and custom code.

Microsoft Azure provides multiple services that you can use to create AI solutions. However, before embarking on an AI application development project, it's useful to consider the available options for services, tools, and frameworks as well as some principles and practices that can help you succeed.

This module explores some of the key considerations for planning an AI development project, and introduces Azure AI Foundry; a comprehensive platform for AI development on Microsoft Azure."""
    }
]

search_index_client = SearchClient(endpoint=azure_search_endpoint, index_name="exam-simulator-idx", credential=credential)

result = search_index_client.upload_documents(documents=documents)

In [14]:
results = search_index_client.search(
    query_type="simple",
    search_text="*",
    select = 'Module,Topic,content',
    include_total_count=True
)

for k in results:
    print(k['Topic'])
    print(k['Module'])
    print(k['@search.score'])
    print(k['content'])

Plan and prepare to develop AI solutions on Azure
Develop generative AI apps in Azure
1.0
Introduction
Completed
100 XP
1 minute
The growth in the use of artificial intelligence (AI) in general, and generative AI in particular means that developers are increasingly required to create comprehensive AI solutions. These solutions need to combine machine learning models, AI services, prompt engineering solutions, and custom code.

Microsoft Azure provides multiple services that you can use to create AI solutions. However, before embarking on an AI application development project, it's useful to consider the available options for services, tools, and frameworks as well as some principles and practices that can help you succeed.

This module explores some of the key considerations for planning an AI development project, and introduces Azure AI Foundry; a comprehensive platform for AI development on Microsoft Azure.
